# Quick tutorial/example, using Env. Data Sci example if possible!

- Okay to incorporate examples found elsewhere (provide link and acknowledgement!)

In [62]:
# import libraries 
import pandas as pd
import numpy as np


In [78]:
# choose data
olympic_athletes = pd.read_csv('../data/Olympic_Athletes/athlete_events.csv')

# make subset of data with only the medaling teams
olympic_subset = olympic_athletes[
    ['Name', 'Team', 'Year', 'Event', 'Medal']
].dropna() # drop rows with na values

# # group the data 
# by_team = olympic_subset.groupby([pd.Grouper(key = 'Year'),
#                                  pd.Grouper(key = 'Medal'),
#                                  pd.Grouper(key = 'Team')]).count()

# by_team.head(10)

# select one country
china_medals = olympic_subset.loc[olympic_subset['Team'] == 'China'].groupby([pd.Grouper(key = 'Year'),
                                                                              pd.Grouper(key = 'Medal')]).count()

china_pivot = china_medals.pivot_table(index = 'Year',
                                       columns = 'Medal',
                                       values = 'Name')

china_pivot.head(10)


Medal,Bronze,Gold,Silver
Year,,,
1984,37.0,24.0,13.0
1988,30.0,4.0,16.0
1992,15.0,14.0,44.0
1994,2.0,NaN,1.0
1996,15.0,13.0,66.0
1998,5.0,NaN,9.0
2000,15.0,31.0,19.0
2002,7.0,2.0,5.0
2004,13.0,42.0,27.0


In [61]:
# to use the bokeh library start with this line
from bokeh.plotting import figure, show

# create a new plot with a title and axis labels
p = figure(title="Medals Won at the Olympics by China", 
           x_axis_label='Year', 
           y_axis_label='Number of Medals')

# plotting the three lines
p.line(x=china_pivot.index, 
       y=china_pivot.Gold, legend_label=str('Gold'))
p.line(x=china_pivot.index, 
       y=china_pivot.Silver, legend_label=str('Silver'))
p.line(x=china_pivot.index, 
       y=china_pivot.Bronze, legend_label=str('Bronze'))

show(p)


In [103]:
# # to use the bokeh library start with this line
from bokeh.plotting import figure, show
from bokeh.models import SingleIntervalTicker, LinearAxis

# figure = china_pivot['Bronze']
# figure.plot(kind='hist', 
#             title='Distribution of Minutes')

# create figure
p = figure(title="Medals Won at the Olympics by China", 
           x_axis_label='Year', 
           y_axis_label='Number of Medals',
           x_axis_type = None) # remove defalt x-axis
  
# add a circle renderer with
# size, color and alpha
p.circle(china_pivot.index, china_pivot['Gold'], 
         size = 10, color = "navy", alpha = 0.5)
p.circle(china_pivot.index, china_pivot['Silver'], 
         size = 10, color = "pink", alpha = 0.5)
p.circle(china_pivot.index, china_pivot['Bronze'], 
         size = 10, color = "orange", alpha = 0.5)

# change axis ticks
ticker = SingleIntervalTicker(interval = 2, # make year interval on new axis 2
                              num_minor_ticks = 0) # set number of ticks between major ticks
xaxis = LinearAxis(ticker = ticker)
p.add_layout(xaxis, 'below') # identify where new axis should lay
  
# show the results
show(p) 